# Data Understanding

In [939]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
import seaborn as sns


In [940]:
df_sessions = pd.read_csv('data/ga_sessions.csv')
df_hits = pd.read_csv('data/ga_hits.csv')

/var/folders/3s/h4tbdn8n3bg8g9r44dkph73w0000gn/T/ipykernel_29248/2144210169.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sessions = pd.read_csv('data/ga_sessions.csv')


In [941]:
df_sessions.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.163775,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.162225,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.162225,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [942]:
df_hits.head()

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN


In [943]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 18 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              int64 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
dtypes: int64(1), object(17)
memory usage: 255.4+ MB


In [944]:
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_type        object 
 5   hit_referer     object 
 6   hit_page_path   object 
 7   event_category  object 
 8   event_action    object 
 9   event_label     object 
 10  event_value     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.3+ GB


In [945]:
# соеденим две таблицы в одну по общим признакам

In [946]:
merged_df = pd.merge(df_sessions, df_hits, left_on=['session_id', 'visit_date'], right_on=['session_id', 'hit_date'], how='inner')

In [947]:
# удалим дубликаты

In [948]:
merged_df = merged_df.drop_duplicates(subset=['session_id', 'hit_date'], keep='first')

In [949]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1732266 entries, 0 to 15685206
Data columns (total 28 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   client_id                 object 
 2   visit_date                object 
 3   visit_time                object 
 4   visit_number              int64  
 5   utm_source                object 
 6   utm_medium                object 
 7   utm_campaign              object 
 8   utm_adcontent             object 
 9   utm_keyword               object 
 10  device_category           object 
 11  device_os                 object 
 12  device_brand              object 
 13  device_model              object 
 14  device_screen_resolution  object 
 15  device_browser            object 
 16  geo_country               object 
 17  geo_city                  object 
 18  hit_date                  object 
 19  hit_time                  float64
 20  hit_number                in

In [950]:
merged_df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,9055434745589932991.1637753792.1637753792,2108382700.163775,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-11-24,3665.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
2,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,...,2021-11-14,921.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
3,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-28,84155.0,10,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_region,KWTCzSIXzoqUWjfUQMgP,NaN
19,9055447046360770272.1622255328.1622255328,2108385564.162225,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,...,2021-05-29,NaN,2,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,sub_page_view,sub_landing,KclpemfoHstknWHFiLit,NaN
22,9055447046360770272.1622255345.1622255345,2108385564.162225,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,...,2021-05-29,NaN,3,event,HbolMJUevblAbkHClEQa,sberauto.com/cars?city=1&rental_page=rental_on...,quiz,quiz_show,KclpemfoHstknWHFiLit,NaN


# Data Preparation

### Drop columns

In [951]:
if merged_df['visit_date'].equals(merged_df['hit_date']):
    print("Все значения идентичны")
else:
    print("Есть различия")

Все значения идентичны


In [952]:
merged_df = merged_df.drop(columns=['visit_date'])

In [953]:
merged_df.session_id.nunique()

1732266

In [954]:
df_cleaned = merged_df.drop(columns=['visit_number', 'visit_time', 'client_id', 'session_id',
                                           'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path',
                                           'event_category', 'event_label', 'event_value'])

In [955]:
df_cleaned.head()

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,event_action
0,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,sub_landing
2,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,sub_landing
3,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,search_form_region
19,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,sub_landing
22,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,quiz_show


In [957]:
df_cleaned[df_cleaned.event_action == 'sub_car_claim_click']

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,event_action
25890,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,Huawei,NaN,360x720,Chrome,Russia,Moscow,sub_car_claim_click
42969,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,375x667,Safari,Russia,Moscow,sub_car_claim_click
76767,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,320x694,Chrome,Russia,Saint Petersburg,sub_car_claim_click
81974,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari,Russia,Moscow,sub_car_claim_click
94133,bByPQxmDaMXgpHeypKSM,referral,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,Samsung,NaN,412x915,Chrome,Russia,Moscow,sub_car_claim_click
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15571355,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,339x753,Chrome,Russia,Moscow,sub_car_claim_click
15594564,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Xiaomi,NaN,393x873,Chrome,Russia,Moscow,sub_car_claim_click
15634483,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Grozny,sub_car_claim_click
15653984,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,NaN,1280x800,Chrome,Russia,Moscow,sub_car_claim_click


In [ ]:
df_cleaned.info()

### Data cleaning

In [ ]:
# начнем обработку наших дынных

In [ ]:
# удалим дубликаты и сохраним таблицу для дальнейшей работы при моделировании

In [ ]:
df_cleaned[df_cleaned.duplicated()]

In [ ]:
df_cleaned = df_cleaned.drop_duplicates()

In [ ]:
df_cleaned.to_csv("data/df_ready.csv", index=False)

In [ ]:
df_cleaned.info()

In [ ]:
msno.bar(df_cleaned)

In [ ]:
missing_values = ((df_cleaned.isna().sum() / len(df_cleaned)) * 100).sort_values()
print ('Процент пропущенных значений:')
missing_values

In [ ]:
# Удалим неимформативные признаки, где процент пропущенных значений больше 25.

In [ ]:
df_cleaned = df_cleaned.drop(columns=['device_model', 'utm_keyword', 'device_os'])

In [ ]:
missing_values = ((df_cleaned.isna().sum() / len(df_cleaned)) * 100).sort_values()
print ('Процент пропущенных значений:')
missing_values

In [ ]:
def print_useful_rows_info(df) :
    print ('Количество полностью заполненных объектов всей выборки:', len(df.dropna()))
    print ('Процент полностью заполненных объектов из всей выборки:', round(len(df.dropna()) / len(df) * 100, 2))

In [ ]:
print_useful_rows_info(df_cleaned)

In [ ]:
df_delete_cols = df_cleaned.copy()

In [ ]:
# Пропуски все еще присутствуют, но теперь мы их уже будем заполнять 

In [ ]:
df_cleaned.device_brand.value_counts(dropna=False)

In [ ]:
# возможно значения которые пропущенны в device_brand имеются в других колонках

In [ ]:
merged_df[merged_df['device_brand'].isna()][[
    'device_brand',
    'device_os',
    'device_category',
    'device_model',
    'device_screen_resolution',
    'device_browser'
]].head(50)


In [ ]:
# Увы, таковых нет, заполним новым значением other

In [ ]:
df_cleaned.device_brand = df_cleaned.device_brand.fillna('other')
df_cleaned.device_brand.isna().sum()

In [ ]:
print_useful_rows_info(df_cleaned)

In [ ]:
# продолжим заполнять пропуски

In [ ]:
df_cleaned.utm_adcontent.value_counts(dropna=False)

In [ ]:
# заполним новым значением other

In [ ]:
df_cleaned.utm_adcontent = df_cleaned.utm_adcontent.fillna('other')
df_cleaned.utm_adcontent.isna().sum()

In [ ]:
print_useful_rows_info(df_cleaned)

In [ ]:
# остался только признак utm_campaign и utm_source

In [ ]:
df_cleaned.utm_campaign.value_counts(dropna=False)

In [ ]:
# заполним новым значением other

In [ ]:
df_cleaned.utm_campaign = df_cleaned.utm_campaign.fillna('other')
df_cleaned.utm_campaign.isna().sum()

In [ ]:
df_cleaned.utm_source.value_counts(dropna=False)

In [ ]:
# заполним новым значением other

In [ ]:
df_cleaned.utm_source = df_cleaned.utm_source.fillna('other')
df_cleaned.utm_source.isna().sum()

In [ ]:
print_useful_rows_info(df_cleaned)

In [ ]:
df_clean = df_cleaned.copy()

### Filter data

In [ ]:
# обработаем те значения в признаках которые могут быть неимформативными

In [ ]:
# заменим такие уникальные знаяения на "other_*"

In [ ]:
df_cleaned.geo_city.value_counts()

In [ ]:
df_cleaned.geo_city.nunique()

In [ ]:
num_unique_cities = (df_cleaned.geo_city.value_counts() <= 1000).sum()
num_unique_cities

In [ ]:
target_values = [
    'sub_car_claim_click', 'sub_car_claim_submit_click',
    'sub_open_dialog_click', 'sub_custom_question_submit_click',
    'sub_call_number_click', 'sub_callback_submit_click',
    'sub_submit_success', 'sub_car_request_submit_click'
]

df_cleaned['event_action'] = df_cleaned['event_action'].apply(lambda x: 1 if x in target_values else 0)

In [ ]:
df_try = df_cleaned.copy()

In [ ]:
# Считаем количество вхождений каждого города
city_counts = df_try['geo_city'].value_counts()
    
df_try['city_category'] = df_try['geo_city'].apply(lambda x: 'Rare' if city_counts[x] <= 1000 else 'Frequent')
target_mean_by_category = df_try.groupby('city_category')['event_action'].mean().reset_index()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='city_category', y='event_action', data=target_mean_by_category)
plt.ylabel('Frequency of Target Variable (1)')
plt.title('Comparison of Target Variable Frequency Between Rare and Frequent Cities')
plt.show()

In [ ]:
target_mean_by_category

In [ ]:
# судя по проведенному анализу зависимости целевого действия от частоты повторений городов, можно сделать вывод,
# что города кототрые редео встречаются несут за собой негативне целеыое действие

# поэтому я решил избавиться от ункикального наименования городов, которые повторяются меньше 5000 раз, 
# из я все переиминую как other_cities

In [ ]:
df_try.head()

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.geo_city = df_cleaned.geo_city.apply(
    lambda x: 'other_cities' if city_counts[x] <= 1000 else x)

In [ ]:
df_cleaned.geo_country.nunique()

In [ ]:
df_cleaned.geo_country.value_counts().to_list()

In [ ]:
num_unique_cities = (df_cleaned.geo_country.value_counts() <= 500).sum()
num_unique_cities

In [ ]:
# Считаем количество вхождений каждого города
country_counts = df_try['geo_country'].value_counts()
    
df_try['country_category'] = df_try['geo_country'].apply(lambda x: 'Rare' if country_counts[x] <= 500 else 'Frequent')
target_mean_by_category = df_try.groupby('country_category')['event_action'].mean().reset_index()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='country_category', y='event_action', data=target_mean_by_category)
plt.ylabel('Frequency of Target Variable (1)')
plt.title('Comparison of Target Variable Frequency Between Rare and Frequent')
plt.show()

In [ ]:
target_mean_by_category

In [ ]:
df_cleaned.geo_country = df_cleaned.geo_country.apply(
    lambda x: 'other_countries' if country_counts[x] <= 500 else x)

In [ ]:
df_cleaned.geo_country.value_counts()

In [ ]:
# utm_campaign             407
# utm_adcontent            281
# device_category            3
# device_brand             201
# device_browser            55

In [ ]:
df_cleaned.utm_campaign.value_counts().to_list()

In [ ]:
num_unique_cities = (df_cleaned.utm_campaign.value_counts() <= 10).sum()
num_unique_cities

In [ ]:
df_cleaned.utm_campaign.nunique()

In [ ]:
utm_campaign_counts = df_try['utm_campaign'].value_counts()
    
df_try['utm_campaign_category'] = df_try['utm_campaign'].apply(lambda x: 'Rare' if utm_campaign_counts[x] <= 10 else 'Frequent')
target_mean_by_category = df_try.groupby('utm_campaign_category')['event_action'].mean().reset_index()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='utm_campaign_category', y='event_action', data=target_mean_by_category)
plt.ylabel('Frequency of Target Variable (1)')
plt.title('Comparison of Target Variable Frequency Between Rare and Frequent')
plt.show()

In [ ]:
target_mean_by_category

In [ ]:
df_cleaned.utm_campaign = df_cleaned.utm_campaign.apply(
    lambda x: 'other_utm_campaign' if utm_campaign_counts[x] <= 10 else x)

In [ ]:
df_cleaned.utm_campaign.value_counts()

In [ ]:
df_cleaned.utm_adcontent.value_counts().to_list()

In [ ]:
num_unique_cities = (df_cleaned.utm_adcontent.value_counts() <= 1).sum()
num_unique_cities

In [ ]:
df_cleaned.utm_adcontent.nunique()

In [ ]:
utm_adcontent_counts = df_try['utm_adcontent'].value_counts()
    
df_try['utm_adcontent_category'] = df_try['utm_adcontent'].apply(lambda x: 'Rare' if utm_adcontent_counts[x] <= 6 else 'Frequent')
target_mean_by_category = df_try.groupby('utm_adcontent_category')['event_action'].mean().reset_index()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='utm_adcontent_category', y='event_action', data=target_mean_by_category)
plt.ylabel('Frequency of Target Variable (1)')
plt.title('Comparison of Target Variable Frequency Between Rare and Frequent')
plt.show()

In [ ]:
target_mean_by_category

In [ ]:
df_cleaned.utm_adcontent = df_cleaned.utm_adcontent.apply(
    lambda x: 'other_utm_adcontent' if utm_adcontent_counts[x] <= 6 else x)

In [ ]:
df_cleaned.utm_adcontent.value_counts()

In [ ]:
df_cleaned.device_brand.value_counts().to_list()

In [ ]:
num_unique = (df_cleaned.device_brand.value_counts() <= 10).sum()
num_unique

In [ ]:
df_cleaned.device_brand.nunique()

In [ ]:
# Считаем количество вхождений каждого города
device_brand_counts = df_try['device_brand'].value_counts()
    
df_try['device_brand_category'] = df_try['device_brand'].apply(lambda x: 'Rare' if device_brand_counts[x] <= 1 else 'Frequent')
target_mean_by_category = df_try.groupby('device_brand_category')['event_action'].mean().reset_index()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='device_brand_category', y='event_action', data=target_mean_by_category)
plt.ylabel('Frequency of Target Variable (1)')
plt.title('Comparison of Target Variable Frequency Between Rare and Frequent')
plt.show()

In [ ]:
target_mean_by_category

In [ ]:
df_cleaned.device_brand = df_cleaned.device_brand.apply(
    lambda x: 'other_device_brand' if device_brand_counts[x] <= 1 else x)

In [ ]:
df_cleaned.device_brand.value_counts()

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.nunique()

In [ ]:
# тперь уникальных значений в каждой категории стало меньше

In [ ]:
# все пустые значения обработаны и совершены преобразования в колонках 

In [ ]:
df_ready = df_cleaned.copy()

### Data transformation

In [ ]:
df_ready.head()

In [ ]:
df_ready.info()

In [ ]:
df_ready.nunique()

In [ ]:
# преобразуем все данные в категориаьные

In [ ]:
df_ready = df_ready.astype('category')
df_ready.info()

In [ ]:
df_transformed = df_ready.copy()

In [ ]:
# на этом этапе этап EDA закончен

# Feature engineering 

In [ ]:
# начну с того, что создам две новые переменные на основе device_screen_resolution

In [ ]:
df_transformed['device_screen_width'] = df_transformed['device_screen_resolution'].apply(lambda x: int(x.split('x')[0]))

In [ ]:
df_transformed['device_screen_height'] = df_transformed['device_screen_resolution'].apply(
    lambda x: int(x.split('x')[1]) if 'x' in x and len(x.split('x')) == 2 else 'other')

In [ ]:
df_transformed.head()

In [ ]:
df_transformed.info()

In [ ]:
# удалим уже ненужный признак device_screen_resolution

In [ ]:
df_transformed = df_transformed.drop(columns=['device_screen_resolution'])

In [ ]:
df_transformed.info()

In [ ]:
df_transformed.utm_medium.value_counts()

In [ ]:
# создадим две новые фичи на основе utm_medium, где ('organic', 'referral', '(none)') это органический трафик
# соответственно остальное это платный трафик

In [ ]:
df_transformed['free_trafic'] = df_transformed.utm_medium.apply(
    lambda x: 1 if x in ['organic', 'referral', '(none)'] else 0)

In [ ]:
df_transformed['payed_trafic'] = df_transformed.utm_medium.apply(
    lambda x: 1 if x not in ['organic', 'referral', '(none)'] else 0)

In [ ]:
df_transformed.info()

In [ ]:
df_transformed.head()

In [ ]:
# создадим две новые фичи на основе по трафику из рекламы в социальных сетях и наоборот (utm_source)

In [ ]:
df_transformed['soc_mdeia_adv'] = df_transformed.utm_source.apply(
    lambda x: 1 if x in ('QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt',
                         'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo',
                         'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm') else 0)

In [ ]:
df_transformed['other_adv'] = df_transformed.utm_source.apply(
    lambda x: 1 if x not in ('QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt',
                         'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo',
                         'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm') else 0)

In [ ]:
df_transformed.head()

In [ ]:
df_transformed.info()

In [ ]:
df_transformed.nunique()

In [ ]:
# избавимся от лишних колонок

In [ ]:
df_transformed = df_transformed.drop(columns=['utm_source', 'utm_medium'])

In [ ]:
df_transformed.info()

In [ ]:
df_transformed.event_action.value_counts()